# ETL Project

In [2]:
#pip install --upgrade snowflake-connector-python

  Using cached snowflake_connector_python-3.12.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (64 kB)
  Using cached asn1crypto-1.5.1-py2.py3-none-any.whl.metadata (13 kB)
  Using cached tomlkit-0.13.2-py3-none-any.whl.metadata (2.7 kB)
Using cached snowflake_connector_python-3.12.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.5 MB)
Using cached asn1crypto-1.5.1-py2.py3-none-any.whl (105 kB)
Using cached tomlkit-0.13.2-py3-none-any.whl (37 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import snowflake.connector

conn = snowflake.connector.connect(
    user='roguchi',
    password='Rsm123123',
    account='nchcfow-qtb66258'
)
cs = conn.cursor()
# Part 2

# Deliverable

## Question 1

In [4]:
from pathlib import Path

In [5]:
import pandas as pd

In [6]:
cs.execute("CREATE DATABASE IF NOT EXISTS maindb") # Creating Database, Stage for PO CSV Files, and Use Public Schema
cs.execute('USE DATABASE maindb')
cs.execute('CREATE OR REPLACE STAGE POStage')
cs.execute('USE SCHEMA PUBLIC')

In [7]:
file_path = Path('Data/Monthly_PO_Data') # Iterates Through Monthly_PO_Data Folder to PUT CSV files for staging
for fp in file_path.iterdir():
    if fp.is_file():
        cs.execute(f"PUT file://{fp} @POStage")
    
print('Done')

Done


In [10]:
cs.execute(
    "CREATE OR REPLACE TABLE PO_Data("
    "PurchaseOrderID STRING, "
    "SupplierID STRING, "
    "OrderDate date, "
    "PurchaseOrderLineID integer,"
    "StockItemID integer, "
    "ReceivedOuters integer, "
    "ExpectedUnitPricePerOuter float, "
    "LastReceiptDate date, "
    "IsOrderLineFinalized boolean)"
) # Create PO_Data Table to house all CSV PO Data

In [11]:

cs.execute("""
COPY INTO PO_Data (PurchaseOrderID, SupplierID, OrderDate, PurchaseOrderLineID, StockItemID, ReceivedOuters, ExpectedUnitPricePerOuter, LastReceiptDate, IsOrderLineFinalized)
FROM (SELECT $1, $2, $3, $13, $14, $17, $19, $20, $21 FROM @POStage)
FILE_FORMAT = (
    TYPE = 'CSV',
    FIELD_OPTIONALLY_ENCLOSED_BY = '"',
    TIMESTAMP_FORMAT = 'MM/DD/YYYY HH24:MI'
    SKIP_HEADER = 1
)
ON_ERROR = 'CONTINUE'
""") # Select only relevant columns from the PO Data, While Copying in Data From Staged CSV Files


## Question 2

In [18]:
cs.execute("ALTER TABLE PO_Data ADD COLUMN POAmount FLOAT;") # Create new column POAmount From calcuations
cs.execute("UPDATE PO_Data SET POAmount = ReceivedOuters * ExpectedUnitPricePerOuter") # Update PO_Data Table with said calcuations
cs.execute("""CREATE OR REPLACE TABLE PO_Data AS
           SELECT PURCHASEORDERID, SupplierID, OrderDate, sum(POAMOUNT) as POAMOUNT FROM PO_DATA
           GROUP BY PURCHASEORDERID, SUPPLIERID, ORDERDATE;
           """) ### Perform an Aggregation ON POAmount to do sum per line item

## Question 3

In [19]:
cs.execute("CREATE OR REPLACE STAGE xml_stage") # Create stage for XML File

In [44]:
xml_path = Path("Supplier_Transactions_XML.xml")
cs.execute(f"PUT file://Data/{xml_path} @xml_stage") # PUT XML file into new stage

In [45]:
cs.execute("""CREATE OR REPLACE TABLE xml_staging_table (
    raw_data VARIANT
);""") # Create Variant Table where XML file contents are stored in

In [46]:
cs.execute("""COPY INTO xml_staging_table
        FROM @xml_stage
        FILE_FORMAT = (TYPE = 'XML')
""") # Copy said XML data into the table that was created in the cell above

In [47]:
cs.execute("""CREATE OR REPLACE TABLE supplier_transactions (
    supplier_transaction_id STRING,
    supplier_id STRING,
    purchase_order_id STRING,
    supplier_invoice_number STRING,
    transaction_date STRING,
    amount_excluding_tax float,
    tax_amount float,
    transaction_amount float,
    outstanding_balance float,
    finalization_date STRING,
    is_finalized INTEGER
);""") # Create a table where Shreded XML data will be housed with relevant columns

In [48]:
cs.execute("""INSERT INTO supplier_transactions
SELECT
    XMLGET(flattened.value, 'SupplierTransactionID'):"$"::STRING as SupplierTransactionID,
    XMLGET(flattened.value, 'SupplierID'):"$"::STRING as SupplierID,
    XMLGET(flattened.value, 'PurchaseOrderID'):"$"::STRING as PurchaseOrderID,
    XMLGET(flattened.value, 'SupplierInvoiceNumber'):"$"::STRING as SupplierInvoiceNumber,
    XMLGET(flattened.value, 'TransactionDate'):"$"::STRING as TransactionDate,
    XMLGET(flattened.value, 'AmountExcludingTax'):"$"::FLOAT as AmountExcludingTax,
    XMLGET(flattened.value, 'TaxAmount'):"$"::FLOAT as TaxAmount,
    XMLGET(flattened.value, 'TransactionAmount'):"$"::FLOAT as TransactionAmount,
    XMLGET(flattened.value, 'OutstandingBalance'):"$"::FLOAT as OutstandingBalance,
    XMLGET(flattened.value, 'FinalizationDate'):"$"::STRING as FinalizationDate,
    XMLGET(flattened.value, 'IsFinalized'):"$"::INTEGER as IsFinalized,
FROM 
    XML_STAGING_TABLE,
    LATERAL FLATTEN(input => xml_staging_table.raw_data:"$") AS flattened
WHERE 
    GET(flattened.value, '@') = 'row';

""") # Shred XML data and house into the table that was created in the cell above

## Question 4 + 5

In [49]:
cs.execute("""
CREATE OR REPLACE TABLE Purchase_Orders_And_Invoices_Table AS (
SELECT *, AMOUNT_EXCLUDING_TAX - POAMOUNT AS Invoiced_vs_Quoted FROM SUPPLIER_TRANSACTIONS AS A
JOIN PO_DATA AS B
ON B.PURCHASEORDERID = A.PURCHASE_ORDER_ID);
""") # Joined Shreded XML table with PO_Data Table on PurchaseOrderID and create new calculated field under POAmount
#This creates 0 across the two columns

In [50]:
cs.execute("Create OR REPLACE Materialized VIEW Purchase_Orders_And_Invoices AS (SELECT * FROM Purchase_Orders_And_Invoices_Table)")
# Create materialized view from new table above

## Question 6

In [51]:
import psycopg2

def execute_pgsql_file(file_path):
    # Connect to PostgreSQL
    conn = psycopg2.connect(
        dbname="rsm-docker",
        user="jovyan",
        password="postgres",
        host="127.0.0.1",
        port="8765"
    )
    
    cursor = conn.cursor()

    # Open and execute the .pgsql file
    with open(file_path, 'r') as f:
        cursor.execute(f.read())

    conn.commit()
    cursor.close()
    conn.close()

# Example usage
pgsql_fp = Path('Data/supplier_case.pgsql')
execute_pgsql_file(pgsql_fp) #Parse through PGSQL File to read contents of said File


In [52]:
def export_data_to_csv():
    conn = psycopg2.connect(
        dbname="rsm-docker",
        user="jovyan",
        password="postgres",
        host="127.0.0.1",
        port="8765"
    )
    
    cursor = conn.cursor()
    export_query = "COPY (SELECT * FROM supplier_case) TO STDOUT WITH CSV HEADER"
    
    with open('Data/supplier_case.csv', 'w') as f:
        cursor.copy_expert(export_query, f)
    
    cursor.close()
    conn.close()

export_data_to_csv()
#COPY Table created from PGSQL Script into a CSV File

In [53]:
cs.execute('CREATE OR REPLACE STAGE Supplier_Case')
cs.execute('USE SCHEMA public')
# Create a stage for newly created CSV File from the PGSQL Script

In [54]:
def upload_to_snowflake(csv_file_path):
    conn = snowflake.connector.connect(
        user='roguchi',
        password='Rsm123123',
        account='nchcfow-qtb66258',
        database='maindb',
        schema='public'
    )
    
    cursor = conn.cursor()
    put_command = f"PUT file://{csv_file_path} @Supplier_Case"
    cursor.execute(put_command)

    cursor.close()
    conn.close()

csv_fp = Path('Data/supplier_case.csv')
upload_to_snowflake(csv_fp)

# Put CSV file for staging


In [55]:
def create_and_load_table_in_snowflake(create_table_statement):
    conn = snowflake.connector.connect(
        user='roguchi',
        password='Rsm123123',
        account='nchcfow-qtb66258',
        database='maindb',
        schema='Public'
    )
    
    cursor = conn.cursor()
    cursor.execute(create_table_statement)
    
    copy_command = """COPY INTO supplier_case (SupplierID, SupplierName, PostalCityID, Phonenumber, Deliveryaddressline1, DeliveryAddressLine2,
    DeliveryPostalCode, DeliveryLocation, PostalAddressLine1, PostalAddressLine2, PostalPostalCode)
                      FROM (SELECT $1, $2, $7, $16, $19, $20, $21, $22, $23, $24, $25 FROM @Supplier_case)
                        FILE_FORMAT=(TYPE=CSV,
                                    FIELD_OPTIONALLY_ENCLOSED_BY = '"',
                                    TIMESTAMP_FORMAT = 'MM/DD/YYYY HH24:MI'
                                    SKIP_HEADER = 1
                                    )
                        ON_ERROR = 'CONTINUE'"""
    # Copy the CSV file contents with relevant columns into Supplier_case table
    cursor.execute(copy_command)
    
    cursor.close()
    conn.close()

create_table = ("""
                CREATE OR REPLACE TABLE Supplier_Case (
                supplierid INTEGER,
                suppliername STRING,
                postalcityid INTEGER,
                phonenumber STRING,
                deliveryaddressline1 STRING,
                deliveryaddressline2 STRING,
                DeliveryPostalCode INTEGER,
                deliverylocation STRING,
                postaladdressline1 STRING,
                postaladdressline2 STRING,
                postalpostalcode INTEGER
                )
                """)
    # SQL Script for creating table where CSV contents are later copied into


create_and_load_table_in_snowflake(create_table)


## Question 7

In [56]:
cs.execute("CREATE OR REPLACE STAGE txtstage") # Make Stage for TXT file

In [57]:
txt_path = Path("2021_Gaz_zcta_national.txt")
cs.execute(f"PUT file://Data/{txt_path} @txtstage") # Put TXT File for staging

In [58]:
cs.execute("""CREATE OR REPLACE TABLE zcta_geolocation (
    GEOID VARCHAR(5),
    INTPTLAT FLOAT,
    INTPTLONG FLOAT
);
""") # Create table for TXT file

In [59]:
cs.execute("""
COPY INTO zcta_geolocation (GEOID, INTPTLAT, INTPTLONG)
FROM (SELECT $1, $6, $7 FROM @txtstage)
FILE_FORMAT = (
    TYPE = 'CSV',
    FIELD_OPTIONALLY_ENCLOSED_BY = '"',
    FIELD_DELIMITER = '\t',
    SKIP_HEADER = 1
)
""") # Copy in contents for TXT file into the newly created table from the cell above

In [60]:
cs.execute("""
        CREATE OR REPLACE TABLE supplier_zip_geolocation AS (
        SELECT sc.postalpostalcode, z.INTPTLAT AS latitude, z.INTPTLONG AS longitude
        FROM supplier_case sc
        JOIN zcta_geolocation z ON sc.postalpostalcode = z.GEOID
        GROUP BY sc.postalpostalcode, z.INTPTLAT, z.INTPTLONG);
""") # Tie each supplier address to a zip code to have matching Longitude & Latitude

In [61]:
cs.execute("""
CREATE OR REPLACE TABLE zip_weather_station_mapping AS
    SELECT 
        sz.postalpostalcode,
        ws.noaa_weather_station_id AS station_id,
        ws.noaa_weather_station_name AS station_name,
        st_distance(
            st_makepoint(sz.longitude, sz.latitude),
            st_makepoint(ws.longitude, ws.latitude)
        ) AS distance
    FROM maindb.public.supplier_zip_geolocation sz
    JOIN WEATHER__ENVIRONMENT.cybersyn.noaa_weather_station_index ws
    ON st_distance(st_makepoint(sz.longitude, sz.latitude), st_makepoint(ws.longitude, ws.latitude)) < 50000
    QUALIFY ROW_NUMBER() OVER (PARTITION BY sz.postalpostalcode ORDER BY distance ASC) = 1;
""") # Calculate qualified zip codes that are close enough to a weather station by using Latitude/Longitude

In [62]:
cs.execute("""
CREATE OR REPLACE TABLE zip_code_weather_data AS
           SELECT 
            z.postalpostalcode,
            wm.date,
            wm.value as Max_Temp,
            wm.variable
            FROM maindb.public.zip_weather_station_mapping z
            JOIN WEATHER__ENVIRONMENT.cybersyn.noaa_weather_metrics_timeseries wm
            ON z.station_id = wm.noaa_weather_station_id
            WHERE wm.variable = 'maximum_temperature';
""") # Tie time series maximum temperature weather readings to qualified weather_stations/zipcodes 

In [63]:
cs.execute("""
CREATE OR REPLACE TABLE supplier_zip_code_weather_table AS 
SELECT A.PostalPostalCode, B.date, B.Max_temp FROM supplier_case as A
JOIN zip_code_weather_data as B ON A.postalpostalcode = B.postalpostalcode
ORDER BY DATE;
""") # Join the Supplier Zip Codes with the time series data created in the cell above

In [64]:
cs.execute("CREATE OR REPLACE MATERIALIZED VIEW supplier_zip_code_weather AS (SELECT * FROM supplier_zip_code_weather_table);")
# Create materialized view from table above

## Question 8

In [66]:
cs.execute("""CREATE OR REPLACE VIEW order_zip_temp AS 
SELECT A.*, C.SupplierName, C.PostalCityID, C.Phonenumber, C.DELIVERYADDRESSLINE2, C.DeliveryPostalCode, C.DeliveryLocation, C.PostalAddressLine1
FROM supplier_zip_code_weather AS A
LEFT JOIN purchase_orders_and_invoices AS B ON A.DATE = B.transaction_date
LEFT JOIN supplier_case AS C ON A.postalpostalcode = C.postalpostalcode
WHERE A.max_temp is NOT NULL;""")
# Tie all supplier information, transaction information, time series weather data into 1 table by joining on zipcode and dates
# This returns only 1 zipcode as this is the only one with 'high temperature' readings